<a href="https://colab.research.google.com/github/sarv936/bigdata-taxi-pipeline/blob/main/bigdata_taxi_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, hour, dayofweek


In [3]:
spark = SparkSession.builder \
    .appName("NYC Taxi ETL Pipeline") \
    .getOrCreate()

In [7]:
from google.colab import files
uploaded = files.upload()

Saving yellow_tripdata_sample.csv to yellow_tripdata_sample.csv


In [8]:
df = spark.read.csv("yellow_tripdata_sample.csv", header=True, inferSchema=True)

In [9]:
df_clean = df.filter((col("passenger_count") > 0) & (col("trip_distance") > 0))


In [10]:
df_features = df_clean.withColumn("pickup_hour", hour("tpep_pickup_datetime")) \
                      .withColumn("pickup_day", dayofweek("tpep_pickup_datetime"))

In [11]:
agg = df_features.groupBy("pickup_day", "pickup_hour") \
    .avg("trip_distance", "fare_amount") \
    .withColumnRenamed("avg(trip_distance)", "avg_distance") \
    .withColumnRenamed("avg(fare_amount)", "avg_fare")

agg.show(10)

+----------+-----------+-----------------+------------------+
|pickup_day|pickup_hour|     avg_distance|          avg_fare|
+----------+-----------+-----------------+------------------+
|         6|          0|3.222272727272728|12.818181818181818|
+----------+-----------+-----------------+------------------+



In [12]:
agg.write.mode("overwrite").csv("output/aggregated_trips")

In [13]:
!zip -r aggregated_trips.zip output/
from google.colab import files
files.download("aggregated_trips.zip")

  adding: output/ (stored 0%)
  adding: output/aggregated_trips/ (stored 0%)
  adding: output/aggregated_trips/part-00000-ecb71550-b317-4ed8-aa83-8b9b74e22baa-c000.csv (deflated 46%)
  adding: output/aggregated_trips/._SUCCESS.crc (stored 0%)
  adding: output/aggregated_trips/_SUCCESS (stored 0%)
  adding: output/aggregated_trips/.part-00000-ecb71550-b317-4ed8-aa83-8b9b74e22baa-c000.csv.crc (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>